In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# handle gpu leakage issue on DSI cluster
# ideally we shouldn't have to do this but leaving it for now in case the issue pops up again
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from typing import List, Optional, Tuple, Union

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, LlamaTokenizer
from transformers.modeling_outputs import CausalLMOutputWithPast

import scripts

In [4]:
# Confirm GPUs are working properly and set default device explicitly
device = "cuda:0"
torch.cuda.device_count()

1

In [148]:
## References ##
# Huggingface docs: https://huggingface.co/docs/transformers/main/model_doc/llama#transformers.LlamaForCausalLM 

## modeling_llama.py ##
# LlamaForCausalLM » https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py#L989 
# forward » https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py#L989 

## utils.py ##
# GenerationMixin » https://github.com/huggingface/transformers/blob/main/src/transformers/generation/utils.py#L571
# generate » https://github.com/huggingface/transformers/blob/main/src/transformers/generation/utils.py#L1432
# greedy_search » https://github.com/huggingface/transformers/blob/main/src/transformers/generation/utils.py#L2447 
# actual forward pass » https://github.com/huggingface/transformers/blob/main/src/transformers/generation/utils.py#L2614 

class EditableLlama(LlamaForCausalLM):
    def __init__(self, config, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.edit_weights = None
        self.edit_bias = None
        self.edit_scalar = None
        self.edit_count = 0

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, CausalLMOutputWithPast]:
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        hidden_states = outputs[0]

        # TODO: maybe this should actually take the model dtype if I can get it
        # Also, should I edit every token?
        # torch.Size([1, 11, 4096])
        # torch.Size([1, 1, 4096])
        if self.edit is not None:
            edit_norm = self.edit / torch.norm(self.edit)
            # TODO: wait...do I want the final token or the first token?
            projection = edit_norm * torch.dot(hidden_states[:,-1,:].squeeze(0), edit_norm)
            print(projection.shape)
            hidden_states = hidden_states - projection
            print(hidden_states.shape)
            # TODO: renormalize?

        if self.config.pretraining_tp > 1:
            lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)
            logits = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]
            logits = torch.cat(logits, dim=-1)
        else:
            logits = self.lm_head(hidden_states)
        logits = logits.float()

        loss = None
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = CrossEntropyLoss()
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # Enable model parallelism
            shift_labels = shift_labels.to(shift_logits.device)
            loss = loss_fct(shift_logits, shift_labels)

        if not return_dict:
            output = (logits,) + outputs[1:]
            return (loss,) + output if loss is not None else output

        return CausalLMOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def generate_with_edit(self, inputs, edit_weights=None, edit_bias=None, edit_scalar=None, **generate_kwargs):
        self.edit_weights = edit_weights
        self.edit_bias = edit_bias
        self.edit_scalar = edit_scalar
        self.edit_count = 0
        return self.generate(inputs, **generate_kwargs)

In [6]:
MODEL_DIR = "/net/projects/veitch/LLMs/"
MODEL = "llama1-based-models/alpaca-7b" # or llama2-based-models/llama2-hf/Llama-2-7b-chat-hf
MODEL_PATH = MODEL_DIR + MODEL

In [ ]:
# TODO: Clean this up
# This cell is for doing ROME edits
# ROME model and tokenizer
# MODEL_NAME = "gpt2-xl"
# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer.pad_token = tokenizer.eos_token

In [133]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [149]:
# model = LlamaForCausalLM.from_pretrained(MODEL_PATH, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto").to(device)
model = EditableLlama.from_pretrained(MODEL_PATH, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto").to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
LABELS = "title" # choices are "title" and "categories"
DATA_FOLDER = "data/"
TRAIN_FILEPATH = DATA_FOLDER + "train-12-articles.csv"
VAL_FILEPATH = DATA_FOLDER + "val-12-articles.csv"
PROBE_TYPE = "linear" # choices are "linear" and "mlp"
LAYER = None
EPOCHS = 50
BATCH_SIZE = 4
AGGREGATION = "max" # choices are "max" or "mean"
PRINT_PROGRESS = True

In [10]:
train_data, label_encoder_title, label_encoder_cats = scripts.get_hidden_states(TRAIN_FILEPATH, model, tokenizer, device)
val_data, _, _ = scripts.get_hidden_states(VAL_FILEPATH, model, tokenizer, device)

In [11]:
# to use entire flattened activations use layer = None
train_dataset = scripts.create_dataset(*train_data, layer=-1)

In [12]:
val_dataset = scripts.create_dataset(*val_data, layer=-1)

In [13]:
val_dataset.Xs[0].shape

torch.Size([4096])

In [14]:
probe, _ = scripts.train_handler(train_dataset, val_dataset, label_encoder_title, probe_type=PROBE_TYPE, labels=LABELS, batch_size=BATCH_SIZE, epochs=EPOCHS, print_progress=PRINT_PROGRESS)

[Training][5] loss: 1.400
[Validation][5] loss: 0.055
[Validation]5 accuracy: 0.917
[Training][10] loss: 1.884
[Validation][10] loss: 2.103
[Validation]10 accuracy: 0.861
[Training][15] loss: 0.464
[Validation][15] loss: 0.280
[Validation]15 accuracy: 0.917
[Training][20] loss: 0.000
[Validation][20] loss: 0.184
[Validation]20 accuracy: 0.917
[Training][25] loss: 0.000
[Validation][25] loss: 0.167
[Validation]25 accuracy: 0.917
[Training][30] loss: 0.000
[Validation][30] loss: 0.158
[Validation]30 accuracy: 0.917
[Training][35] loss: 0.000
[Validation][35] loss: 0.146
[Validation]35 accuracy: 0.917
[Training][40] loss: 0.000
[Validation][40] loss: 0.148
[Validation]40 accuracy: 0.917
[Training][45] loss: 0.000
[Validation][45] loss: 0.147
[Validation]45 accuracy: 0.917
text:  the technique of photographing successive drawings or positions of puppets or models to create an illusion of movement when the movie is shown as a sequence.
labels:  Animation
predicted labels:  Animation
text:  

### Try Editing With Linear Probe

In [15]:
probe.fc1.weight.data.shape

torch.Size([12, 4096])

In [18]:
probe.fc1.bias

Parameter containing:
tensor([-0.0066,  0.0050, -0.0160,  0.0071, -0.0330, -0.0091, -0.0022,  0.0082,
         0.0081,  0.0164,  0.0045,  0.0345], requires_grad=True)

In [281]:
def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, CausalLMOutputWithPast]:
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        hidden_states = outputs[0]

        # TODO: maybe this should actually take the model dtype if I can get it
        # Also, should I edit every token?
        # torch.Size([1, 11, 4096])
        # torch.Size([1, 1, 4096])
        # Hack to only edit the first time through
        if self.edit_weights is not None and self.edit_count == 0:
            # edit_norm = self.edit / torch.norm(self.edit)
            # # TODO: wait...do I want the final token or the first token?
            # projection = edit_norm * torch.dot(hidden_states[:,-1,:].squeeze(0), edit_norm)
            # print(projection.shape)
            # hidden_states = hidden_states - projection
            # print(hidden_states.shape)
            # # TODO: renormalize?

            '''
            This is a crude edit that seems to work if I just use the probe weights
        
            unit_edit = self.edit/torch.norm(self.edit)
            new_hidden_states = []
            for token in hidden_states[0,:,:]:
                new_hidden_states.append(token - torch.dot(token, unit_edit) * unit_edit)
            new_hidden_states = torch.stack(new_hidden_states).unsqueeze(0)

            hidden_states = new_hidden_states
            '''
            weights_normalized = self.edit_weights/torch.norm(self.edit_weights)
            # print("self.edit_weights: ", self.edit_weights)
            print("torch.norm(self.edit_weights): ", torch.norm(self.edit_weights).item())
            # print("weights_normalized: ", weights_normalized)
            # TODO: vectorize this — doing with a for loop to prevent weird projection behavior
            new_hidden_states = []
            for token in hidden_states[0,:,:]:
                distance = torch.abs(torch.dot(self.edit_weights, token) + self.edit_bias)/torch.norm(self.edit_weights)
                # print("distance: ", distance)
                edit_distance = self.edit_scalar * torch.sign(distance) * distance * weights_normalized
                # print("edit_distance norm: ", torch.norm(edit_distance))
                new_hidden_state = token - edit_distance
                # print("new_hidden_state norm: ", torch.norm(new_hidden_state).item())
                new_distance = (torch.dot(self.edit_weights, new_hidden_state) + self.edit_bias)/torch.norm(self.edit_weights)
                if new_distance != float('inf') and new_distance != -float('inf'):
                    print("new_distance: ", new_distance.item())
                # print(f"norm diff: ", (torch.norm(token)-torch.norm(new_hidden_state)).item())
                # Rescale new_hidden_state to have the same norm as the original token
                if torch.norm(new_hidden_state) != 0:
                    new_hidden_state_rescaled = new_hidden_state * (torch.norm(token) / torch.norm(new_hidden_state))
                else:
                    new_hidden_state_rescaled = new_hidden_state
                new_hidden_states.append(new_hidden_state_rescaled)
                # print("Orthogonal? ", torch.dot(new_hidden_state, weights_normalized))
            hidden_states = torch.stack(new_hidden_states, dim=0).unsqueeze(0)
            self.edit_count += 1
            # print("new_hidden_states shape: ", new_hidden_states.shape)

        if self.config.pretraining_tp > 1:
            lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)
            logits = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]
            logits = torch.cat(logits, dim=-1)
        else:
            logits = self.lm_head(hidden_states)
        logits = logits.float()

        loss = None
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = CrossEntropyLoss()
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # Enable model parallelism
            shift_labels = shift_labels.to(shift_logits.device)
            loss = loss_fct(shift_logits, shift_labels)

        if not return_dict:
            output = (logits,) + outputs[1:]
            return (loss,) + output if loss is not None else output

        return CausalLMOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [165]:
for i, lbl in enumerate(label_encoder_title.classes_):
    print(f"{i}: {lbl}")

0: Agriculture
1: Alaska
2: Alchemy
3: Algae
4: American football
5: Amphibian
6: Anarchism
7: Animation
8: Anthropology
9: Appellate court
10: Astronomer
11: Basketball


In [282]:
model.forward = forward.__get__(model, EditableLlama)

In [286]:
prompt = "I hate authority and my favorite form of government is"
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
probe_idx = 6
edit_scalar = 14

In [274]:
prompt = "I love going down to the park and playing some pickup "
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
probe_idx = 11
edit_scalar = 27.5

In [ ]:
prompt = "As a kid I was really interested in looking at the stars. When I grew up I wanted to be "
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
probe_idx = 11
edit_scalar = 27.5

In [284]:
edit_weights = 5 * torch.rand(4096) # random noise
edit_weights = None
edit_weights = probe.fc1.weight.data[probe_idx]
edit_bias = probe.fc1.bias[probe_idx]

edit_weights = edit_weights.half().to(device)
edit_bias = edit_bias.half().to(device)

In [287]:
output = model.generate_with_edit(input_ids, edit_weights=edit, edit_bias=edit_bias, edit_scalar=edit_scalar, max_length=50, num_return_sequences=1, output_hidden_states=False)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
generated_text

torch.norm(self.edit_weights):  2262.0
new_distance:  -8.078125
norm diff:  -0.375
new_distance:  -1.3388671875
norm diff:  -0.0625
new_distance:  -3.26171875
norm diff:  -0.0625
new_distance:  -11.4921875
norm diff:  -0.4375
new_distance:  -17.453125
norm diff:  -1.0625
new_distance:  -14.6015625
norm diff:  -0.75
new_distance:  -4.1875
norm diff:  -0.0625
new_distance:  -17.734375
norm diff:  -1.0625
new_distance:  -10.4375
norm diff:  -0.375
new_distance:  -15.0390625
norm diff:  -0.75
new_distance:  -13.1171875
norm diff:  -0.5625


"I hate authority and my favorite form of government is anarchy.\nI'm a big fan of the idea of a free society, but I'm not sure that anarchy is the way to get there.\nI think that an"

In [261]:
output = model.generate(input_ids, max_length=50, num_return_sequences=1, output_hidden_states=False)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
generated_text

"I love going down to the park and playing some pickup 5-on-5 basketball. I'm not the best player on the court, but I'm definitely not the worst. I'm a solid player, and I"

In [228]:
# Notes:
# If you just add the probe weights multiplied by 100, you get the probe word repeated ie "animation", "anth", "Alaska"
# Interestingly, anarchy seems to give "libert" (as in "liberty") so these are a similar direction
# Subtracting gives "Dictator"...for Alaska
# Gives "Science" for "Anarchy"

'LeBron James is great at the sport of 1-on-1 basketball. He has proven that he can dominate the game, even against the best players in the world. But when it comes to team basketball, he has yet'